[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rezer0dai/TD3_BC/blob/her/td3_bc_her.ipynb)

In [1]:
!git clone https://github.com/rezer0dai/TD3_BC -b her
!git clone https://github.com/sfujim/TD3
    
!git clone https://github.com/qgallouedec/panda-gym

!pip install -e panda-gym

fatal: destination path 'TD3_BC' already exists and is not an empty directory.
fatal: destination path 'TD3' already exists and is not an empty directory.
fatal: destination path 'panda-gym' already exists and is not an empty directory.
Obtaining file:///content/panda-gym
  Attempting uninstall: panda-gym
    Found existing installation: panda-gym 1.1.0
    Can't uninstall 'panda-gym'. No files were found to uninstall.
  Running setup.py develop for panda-gym


In [2]:
import sys

libs = ["TD3_BC", "TD3", "panda-gym"]
for lib in libs:
    sys.path.append(lib)
    sys.path.append("/content/"+lib)

In [3]:
import numpy as np
import torch
import gym
import argparse
import os

import utils
import TD3_BC
import TD3
import OurDDPG
import config

In [4]:
def eval_policy(policy, eval_env, seed, normalize_state, seed_offset=100, eval_episodes=10):
    load_state = lambda obs: obs["observation"].reshape(1,-1)

    eval_env.seed(seed + seed_offset)

    avg_reward = 0.
    for z in range(eval_episodes):
        state, done = eval_env.reset(), False
        while not done:
            state = load_state(state)
            action = policy.select_action(normalize_state(state), train_mode=z < 3)
            state, reward, done, _ = eval_env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f}")
    print("---------------------------------------")
    return avg_reward


In [5]:
import random
from copy import deepcopy

def her(replay_buffer, achieved_goals, her_ratio, n_future, fail_filter, dummy_filter):
    if not len(achieved_goals):
        return False
    leftover = 1 * (replay_buffer.size != len(achieved_goals))
    replay_buffer.ptr = replay_buffer.ptr - len(achieved_goals) - leftover
    replay_buffer.size = replay_buffer.size - len(achieved_goals) - leftover

    if all(np.linalg.norm(achieved_goals[0]- g) < .05 for g in achieved_goals):
        return False

    norm_ind = replay_buffer.ptr

    ep = deepcopy([ (
            replay_buffer.state[replay_buffer.ptr + i + leftover],
            replay_buffer.action[replay_buffer.ptr + i + leftover],
            replay_buffer.next_state[replay_buffer.ptr + i + leftover],
            replay_buffer.reward[replay_buffer.ptr + i + leftover],
            False
            ) for i in range(len(achieved_goals)) ])

    for _ in range(config.HER_PER_EP):
        ep_ = []
        for j, e in enumerate(ep):
            s, a, n, r, d = deepcopy(e)
            goal = random.choice(achieved_goals[j:][:n_future])
            if random.random() < (dummy_filter if (np.linalg.norm(achieved_goals[0] - goal) < .05) else 0.):
                continue
            
            #assert all(achieved_goals[j] == n[:config.GOAL_SIZE]), "A)failed with {} + {} [{}][{}] <{}>".format(
            #    j, len(ep), achieved_goals[j], n[:config.GOAL_SIZE], np.linalg.norm(achieved_goals[j] - n[:config.GOAL_SIZE])
            #    )
            
            s[-config.GOAL_SIZE:] = deepcopy(goal)
            n[-config.GOAL_SIZE:] = deepcopy(goal)
            
            r = -1. * (np.linalg.norm(achieved_goals[j] - goal) > .05)
            if -1 == r and random.random() < fail_filter:
                continue
            replay_buffer.add(s, a, n, r, d)

        if random.random() < her_ratio:
            continue

        for e in ep:
            replay_buffer.add(*e)

    #assert all(replay_buffer.not_done[:replay_buffer.size])
    #print("\n diff", norm_ind, replay_buffer.ptr, replay_buffer.ptr-norm_ind, sum(0 == replay_buffer.reward[norm_ind:replay_buffer.ptr]))

    #assert all(replay_buffer.not_done[:replay_buffer.size])
    replay_buffer.add(s, a, n, r, True)
    if len(replay_buffer.state[norm_ind:replay_buffer.ptr]) > 1:# edge of buffer
        replay_buffer.normalize_state(replay_buffer.state[norm_ind:replay_buffer.ptr], update=True)

    return True

In [ ]:
import random
from open_gym import make_env

if True:#__name__ == "__main__":
    file_name = f"{config.ENV}_{config.SEED}"
    print("---------------------------------------")
    print(f"Policy: , Env: {config.ENV}, Seed: {config.SEED}")
    print("---------------------------------------")

    env = make_env(config.ENV, render=False, colab=True)
    eval_env = make_env(config.ENV, render=True, colab=True)

    # Set seeds
    env.seed(config.SEED)
    env.action_space.seed(config.SEED)
    torch.manual_seed(config.SEED)
    np.random.seed(config.SEED)
    
    state_dim = env.state_size()
    action_dim = env.action_space.shape[0] 
    max_action = float(env.action_space.high[0])

    kwargs = { # let it default for td3 and td3+bc
            "state_dim": state_dim,
            "action_dim": action_dim,
            "max_action": max_action,
            "discount": config.DISCOUNT,
            "tau": config.TAU,
    }

    # Initialize policy
#    policy = TD3.TD3(**kwargs)
    policy = TD3_BC.TD3_BC(**kwargs)
#    policy = OurDDPG.DDPG(**kwargs)

    replay_buffer_a = utils.ReplayBuffer(state_dim, action_dim)
    replay_buffer_c = utils.ReplayBuffer(state_dim, action_dim)
    replay_buffer_d = utils.ReplayBuffer(state_dim, action_dim)
    replay_buffer_e = utils.ReplayBuffer(state_dim, action_dim)
    #replay_buffer.ptr = replay_buffer.size = replay_buffer.max_size - 10

    print("---------------------------------------")
    print(f"Policy TD3+BC+HER: , Env: {config.ENV}, Seed: {config.SEED}, Observation shape: {state_dim}")
    print("---------------------------------------")

    counter = 0
    score = -100

    done = True
    achieved_goals = []
    load_state = lambda obs: obs["observation"].reshape(1,-1)

    t = 0
    add_prev_exp = 0
    total_steps = config.STEPS_PER_EPOCH * config.EPOCHS
    while t < total_steps:
        counter += 1

        if done:
            a = her(replay_buffer_a, achieved_goals, her_ratio=1., fail_filter=1., n_future=len(achieved_goals), dummy_filter=.97)
            _ = her(replay_buffer_e, achieved_goals, her_ratio=1., fail_filter=.5, n_future=3, dummy_filter=.97)
            c = her(replay_buffer_c, achieved_goals, her_ratio=.8, fail_filter=.5, n_future=len(achieved_goals), dummy_filter=.97)
            assert a == c

            if not a:
              replay_buffer_d.size = replay_buffer_d.size - len(achieved_goals)
              replay_buffer_d.ptr = replay_buffer_d.ptr - len(achieved_goals)

            add_prev_exp = a or c
            achieved_goals = []
            mc_w = random.randint(1, 5) if random.random() > .7 else 0.

            state = load_state(env.reset())

        t += add_prev_exp

        if t < config.START_STEPS:# or (-1. == reward and random.random() < mc_w / 10.):
            action = env.action_space.sample()
        else:
            action = policy.select_action(replay_buffer_a.normalize_state(state))

        observation, reward, done, _ = env.step(action)
        next_state = load_state(observation)
        achieved_goals.append(observation["achieved_goal"])

        replay_buffer_a.add(state, action, next_state, reward, done)
        replay_buffer_c.add(state, action, next_state, reward, done)
        if not done: replay_buffer_d.add(state, action, next_state, reward, done)
        replay_buffer_e.add(state, action, next_state, reward, done)

        state = next_state

        if add_prev_exp and t > config.UPDATE_AFTER and 0 == t % config.UPDATE_EVERY:
            for j in range(config.UPDATE_COUNT):
                policy.train_critic(replay_buffer_c, batch_size=config.BATCH_SIZE)
#                policy.train_critic(replay_buffer_c, batch_size=config.BATCH_SIZE)
                policy.train_actor(replay_buffer_d, use_bc=False, batch_size=config.BATCH_SIZE)
                if score < -30. and 1 == j % 2:
                  policy.train_actor(replay_buffer_a, use_bc=True, batch_size=config.BATCH_SIZE)
                elif score > -30. or 0 == j % 2:
                  policy.train_actor(replay_buffer_e, use_bc=True, batch_size=config.BATCH_SIZE)
            policy.polyak()

        
        # Evaluate episode
        if add_prev_exp and (t + 1) % config.EVAL_FREQ == 0:
            print(f"Epochs : {(t + 1) / config.EVAL_FREQ} [#timestamps : {counter}]")
            score = eval_policy(policy, eval_env, config.SEED, replay_buffer_a.normalize_state)
        if score > -10.:
            break

---------------------------------------
Policy: , Env: panda-pusher, Seed: 0
---------------------------------------
---------------------------------------
Policy TD3+BC+HER: , Env: panda-pusher, Seed: 0, Observation shape: 30
---------------------------------------
Epochs : 1.0 [#timestamps : 20532]
---------------------------------------
Evaluation over 10 episodes: -49.000
---------------------------------------
Epochs : 2.0 [#timestamps : 40183]
---------------------------------------
Evaluation over 10 episodes: -49.000
---------------------------------------


In [ ]:
[ eval_policy(policy, eval_env, random.randint(0, 100000), replay_buffer.normalize_state) for _ in range(10) ]

In [ ]:
double trainig learn one with bc one w/o, l2 norm only for w/o bc